In [1]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import pygsheets
import requests
import time

In [34]:
def get_ball_by_ball_links(season_results_link):
    base_address = "https://www.espncricinfo.com"
    page = urllib.request.urlopen(season_results_link)
    soup = BeautifulSoup(page, 'html.parser')
    scores = soup.find('div',{'class':'scoreCollection__content cricket'})
    matches = scores.findAll('div',{'class':'cscore'})
    match_links = []
    for match in matches:
        scorecard_link = match.find('a',{'name':'&lpos=cricket:scoreboard:scorecard'})
        match_address = base_address + scorecard_link["href"]
        try:
            match_overview_page = urllib.request.urlopen(match_address)
            match_overview_soup = BeautifulSoup(match_overview_page, 'html.parser')
            nav_bar = match_overview_soup.find('div',{'class':'tertiary-nav-container'})
            commentary_button = nav_bar.find('a',{'class':'commentary react-router-link'})
            ball_by_ball_link = base_address + commentary_button['href']
            match_links.append(ball_by_ball_link)
        except:
            print(match_address)
    match_links_df = pd.DataFrame()
    match_links_df["Match Links"] = match_links
    return match_links_df

    


In [35]:
matches_df = get_ball_by_ball_links("https://www.espncricinfo.com/scores/series/8052/season/2019/specsavers-county-championship-division-one")
matches_df.to_csv("C:\\Users\\Glenn's pc\\Documents\\cricket data\\ball_by_ball\\match_links.csv",index = False)

In [72]:
def disect_score(score):
    if len(score) == 0:
        first_innings_runs = 0
        first_innings_wickets = 0
        second_innings_runs = 0
        second_innings_wickets = 0
    else:
        if "&" in score:
            first_innings = score.split("&")[0]
            second_innings = score.split("&")[-1]

            if "ov" in first_innings or "f/o" in first_innings:
                first_innings = first_innings.split("(")[0].strip()
            if "/" in first_innings:
                first_innings_runs = int(first_innings.split("/")[0].strip())
                if "d" in first_innings:
                    first_innings_wickets = int(first_innings.split("/")[-1].replace('d','').strip())
                else:
                    first_innings_wickets = int(first_innings.split("/")[-1].strip())     
            else:
                first_innings_runs = int(first_innings.strip())
                first_innings_wickets = 10


            if "forfeit" in second_innings:
                second_innings_runs = 0
                second_innings_wickets = 0
            elif "ov" in second_innings or "f/o" in second_innings:
                second_innings = second_innings.split("(")[0].strip()
                if "/" in second_innings:
                    second_innings_runs = int(second_innings.split("/")[0].strip())
                    if "d" in second_innings:
                        second_innings_wickets = int(second_innings.split("/")[-1].replace('d','').strip())
                    else:
                        second_innings_wickets = int(second_innings.split("/")[-1].strip())
                else:
                    second_innings_runs = int(second_innings.strip())
                    second_innings_wickets = 10
            else:
                if "/" in second_innings:
                    second_innings_runs = int(second_innings.split("/")[0].strip())
                    if "d" in second_innings:
                        second_innings_wickets = int(second_innings.split("/")[-1].replace('d','').strip())
                    else:
                        second_innings_wickets = int(second_innings.split("/")[-1].strip())
                else:
                    second_innings_runs = int(second_innings.strip())
                    second_innings_wickets = 10
        else:
            if "ov" in score or "f/o" in score:
                score = score.split("(")[0].strip()
            if "/" in score:
                first_innings_runs = int(score.split("/")[0].strip())
                second_innings_runs = 0
                if "d" in score:
                    first_innings_wickets = int(score.split("/")[-1].replace('d','').strip())
                else:
                    first_innings_wickets = int(score.split("/")[-1].strip())
                second_innings_wickets = 0
            else:
                first_innings_runs = int(score.strip())
                first_innings_wickets = 10
                second_innings_runs = 0
                second_innings_wickets = 0
                
    return first_innings_runs, first_innings_wickets, second_innings_runs, second_innings_wickets

In [78]:
def disect_results(result, team_one, team_two):
    if "drawn" in result:
        result = "draw"
    elif team_one in result:
        result = team_one
    elif team_two in result:
        result = team_two
    return result

In [90]:
def get_game_results(match_links_file):
    matches_df = pd.read_csv("C:\\Users\\Glenn's pc\\Documents\\cricket data\\ball_by_ball\\match_links.csv")
    game_results_df = pd.DataFrame()
    for link in matches_df["Match Links"]:
        print(link)
        stadiums = {}
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')

        match_details = soup.find('div',{'class':'gp__cricket__gameHeader'})

        team_one_details = match_details.find('li',{'class':'cscore_item cscore_item--home'})
        team_one = team_one_details.find('span',{'class':'cscore_name cscore_name--long'})
        team_one_score = team_one_details.find('div',{'class':'cscore_score'})

        team_two_details = match_details.find('li',{'class':'cscore_item cscore_item--away'})
        team_two = team_two_details.find('span',{'class':'cscore_name cscore_name--long'})
        team_two_score = team_two_details.find('div',{'class':'cscore_score'})

        team_one_innings = disect_score(team_one_score.text.strip())
        team_two_innings = disect_score(team_two_score.text.strip())

        team_one_runs_first_innings = team_one_innings[0]
        team_one_wickets_first_innings = team_two_innings[1]
        team_one_runs_second_innings = team_one_innings[2]
        team_one_wickets_second_innings = team_two_innings[3]
        team_one_runs = team_one_runs_first_innings + team_one_runs_second_innings
        team_one_wickets = team_one_wickets_first_innings + team_one_wickets_second_innings

        team_two_runs_first_innings = team_two_innings[0]
        team_two_wickets_first_innings = team_one_innings[1]
        team_two_runs_second_innings = team_two_innings[2]
        team_two_wickets_second_innings = team_one_innings[3]
        team_two_runs = team_two_runs_first_innings + team_two_runs_second_innings
        team_two_wickets = team_two_wickets_first_innings + team_two_wickets_second_innings

        game_overview = soup.find('div',{'class':'cscore_info-overview'})
        date = game_overview.text.split(',')[-1].strip()
        location = game_overview.text.split(',')[0].split('at')[-1].strip()
        competition = game_overview.text.split(',')[0].split('at')[0].strip()

        result_description = soup.find('span',{'class':'cscore_notes_game'})
        winner = disect_results(result_description.text, team_one.text, team_two.text)
        game_results_df = game_results_df.append({"Competition":competition,"Location":location,"Date":date,
                                                 "Team One":team_one.text,"Team Two":team_two.text,
                                                 "Team One First Innings Runs":team_one_runs_first_innings,
                                                 "Team One Second Innings Runs":team_one_runs_second_innings,
                                                 "Team One Total Runs":team_one_runs,
                                                 "Team One First Innings Wickets":team_one_wickets_first_innings,
                                                 "Team One Second Innings Wickets":team_one_wickets_second_innings,
                                                 "Team One Wickets":team_one_wickets,
                                                 "Team Two First Innings Runs":team_two_runs_first_innings,
                                                 "Team Two Second Inning Runs":team_two_runs_second_innings,
                                                 "Tem Two Total Runs": team_two_runs,
                                                 "Team Two First Innings Wickets":team_two_wickets_first_innings,
                                                 "Team Two Second Innings Wickets":team_two_wickets_second_innings,
                                                 "Team Two Wickets":team_two_wickets,
                                                 "Result":winner, "Game Link":link},ignore_index=True)
    return game_results_df


In [91]:
game_results_df = get_game_results("C:\\Users\\Glenn's pc\\Documents\\cricket data\\ball_by_ball\\match_links.csv")
game_results_df.to_csv("C:\\Users\\Glenn's pc\\Documents\\cricket data\\ball_by_ball\\game_results.csv")

https://www.espncricinfo.com/series/8052/commentary/1166913/hampshire-vs-essex-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166914/nottinghamshire-vs-yorkshire-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166915/somerset-vs-kent-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166916/hampshire-vs-yorkshire-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166917/nottinghamshire-vs-somerset-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166918/surrey-vs-essex-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166919/warwickshire-vs-kent-specsavers-county-championship-division-one-2019
https://www.espncricinfo.com/series/8052/commentary/1166920/essex-vs-nottinghamshire